# BirdNet sandbox code

This is a small collection of code to manipulate and analyse recording of bird song using the BirdNet Analyzer model to classify bird species based on recordings. The code is intended for use using Google Colab but can be easily altered for use on a local machine.  

In [ ]:
!git clone https://github.com/weharris/birdnetTest

In [ ]:
cd birdnetTest/

In [ ]:
!pip3 install birdnetlib
!pip3 install tflite-runtime
!pip3 install resampy
!pip3 install ffmpeg

In [4]:
from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
from datetime import datetime

## Analyze a single audio file

In [ ]:
# This cell uses BirdNET-Analyzer to analyze a single audio recording.
# Load and initialize the BirdNET-Analyzer models.
analyzer = Analyzer()

testamp = [52.911,-2.4441] # near Market Drayton year=2024, month=4, day=5
xc558223 = [57.4706, -4.163] # Highlands Scotland 2020-05-14 4am year=2020, month=5, day=14
XC553677 = [54.1212, -0.5416] # Ryedale District,(near Helperthorpe) North Yorkshire 2020-05-01 5:10a year=2020, month=5, day=1

recording = Recording(
    analyzer,
    "data/EdH20240405/testamp.mp3",  # path to the recording file
    lat=testamp[0], # latitude of the recording location
    lon=testamp[1], # longitude of the recording location
    date=datetime(year=2024, month=4, day=20), # use date or week_48
    min_conf=0.25,
)
recording.analyze()

myDetections = recording.detections

In [ ]:
# for Single File and list of spp, counts, and mean prediction confidence
import pandas as pd

species_count = {}
species_confidence = {}
for entry in myDetections:
    common_name = entry['common_name']
    confidence = entry['confidence']
    if common_name in species_count:
        species_count[common_name] += 1
        species_confidence[common_name].append(confidence)
    else:
        species_count[common_name] = 1
        species_confidence[common_name] = [confidence]

mean_confidence = {common_name: sum(confidences) / len(confidences) for common_name, confidences in species_confidence.items()}

df = pd.DataFrame({'Common Name': list(species_count.keys()), 'Count': list(species_count.values()), 'Mean Confidence': list(mean_confidence.values())})

sorted_df = df.sort_values('Common Name')
# sorted_df = df.sort_values('Count', ascending=False)
sorted_df.head()

In [ ]:
import matplotlib.pyplot as plt

# Plot the count of each species ID
plt.figure(figsize=(10, 6))  # Set the figure size
plt.barh(df['Common Name'], df['Count'], color='skyblue')  # Create a horizontal bar plot
plt.ylabel('Species')  # Y-axis label
plt.xlabel('Count')  # X-axis label
plt.title('Count of Bird Observations by Species')  # Title of the plot
plt.show()  # Display the plot

In [ ]:
# Plot species by mean confidence
plt.figure(figsize=(10, 6))  # Set the figure size
plt.boxplot(species_confidence.values(), vert=False)  # Create a horizontal boxplot
plt.yticks(range(1, len(species_confidence) + 1), species_confidence.keys())  # Set y-axis labels
plt.ylabel('Species')  # Y-axis label
plt.xlabel('Confidence')  # X-axis label
plt.title('Confidence of Bird Observations by Species')  # Title of the plot

# Annotate the boxplot with species counts
for i, (species, count) in enumerate(species_count.items()):
    plt.text(1.03, i + 1, f'n = {count}', ha='right', va='center')

plt.ylim(0.5, len(species_count) + 0.5)  # Adjust the y-axis limits

plt.subplots_adjust(right=1.1)  # Increase the space in the right side plot margin

plt.show()  # Display the plot

## Sources
https://github.com/kahst/BirdNET-Analyzer?tab=readme-ov-file#62-setup-birdnetlib